In [1]:
cd /data/projects/windturbine-identification/MachineLearningCourse

/data/projects/windturbine-identification/MachineLearningCourse


In [2]:
"""
Created on Mon Feb 18 20:30:48 2019

@author: jschmidt
"""

import os, shutil, random

import pandas as pd
import sys
import gdal
import imp
import numpy as np


import scripts.windturbines.functions_pattern_recognition as fpr
imp.reload(fpr)
from scripts.windturbines.functions_pattern_recognition import get_param
from scripts.windturbines.functions_pattern_recognition import cop_predict
from scripts.windturbines.functions_pattern_recognition import check_image
from scripts.windturbines.functions_pattern_recognition import read_params

Using TensorFlow backend.
/home/jschmidt/.conda/envs/windturbine-identification/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jschmidt/.conda/envs/windturbine-identification/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jschmidt/.conda/envs/windturbine-identification/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 =

EG
13
MA
13
ZA
13
CN
13
IN
13
JP
13
KP
13
PK
13
PH
13
TH
13
CR
13
TR
13
AT
13
BE
13
BG
13
HR
13
DK
13
FI
13
FR
13
DE
13
GR
13
IE
13
IT
13
LT
13
NL
13
NO
13
PL
13
PT
13
RO
13
ES
13
SE
13
GB
13
UA
13
CA
13
MX
13
US
13
AU
13
NZ
13
BR
13
CL
13
UY
13
MIX
13
GLOBAL
13
EG
18
MA
18
ZA
18
CN
18
IN
18
JP
18
KP
18
PK
18
PH
18
TH
18
CR
18
TR
18
AT
18
BE
18
BG
18
HR
18
DK
18
FI
18
FR
18
DE
18
GR
18
IE
18
IT
18
LT
18
NL
18
NO
18
PL
18
PT
18
RO
18
ES
18
SE
18
GB
18
UA
18
CA
18
MX
18
US
18
AU
18
NZ
18
BR
18
CL
18
UY
18
MIX
18
GLOBAL
18
EG
19
MA
19
ZA
19
CN
19
IN
19
JP
19
KP
19
PK
19
PH
19
TH
19
CR
19
TR
19
AT
19
BE
19
BG
19
HR
19
DK
19
FI
19
FR
19
DE
19
GR
19
IE
19
IT
19
LT
19
NL
19
NO
19
PL
19
PT
19
RO
19
ES
19
SE
19
GB
19
UA
19
CA
19
MX
19
US
19
AU
19
NZ
19
BR
19
CL
19
UY
19
MIX
19
GLOBAL
19
EG
13
MA
13
ZA
13
CN
13
IN
13
JP
13
KP
13
PK
13
PH
13
TH
13
CR
13
TR
13
AT
13
BE
13
BG
13
HR
13
DK
13
FI
13
FR
13
DE
13
GR
13
IE
13
IT
13
LT
13
NL
13
NO
13
PL
13
PT
13
RO
13
ES
13
SE
13
GB
13
UA
13
CA
13
MX
13
U

In [5]:
COUNTRY = "US"

train_dir = get_param(COUNTRY,"PATH_ML_IMAGES_TYPE_REGRESSION_TRAIN",18)
test_dir = get_param(COUNTRY,"PATH_ML_IMAGES_TYPE_REGRESSION_TEST",18)

#### delete directories if exist
#### create if not exist

shutil.rmtree(train_dir,ignore_errors=True)
shutil.rmtree(test_dir,ignore_errors=True)

os.makedirs(train_dir)
os.makedirs(test_dir)

src_dir_tb = get_param(COUNTRY,"PATH_RAW_IMAGES_TURBINES")


In [6]:
train_dir

'data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/train/'

In [7]:
turbines_us = pd.read_csv(get_param("US","FILE_TURBINE_LOCATIONS"))


predictions_cs = get_param("US",
                            "PATH_RAW_IMAGES_TURBINES")

turbines_pred =  pd.read_feather(predictions_cs + "all_predictions.feather")


turbines_us['prediction'] = turbines_pred['prediction']
turbines_us['filenames'] = turbines_pred['filenames']

turbines_us['id'] = np.arange(1, turbines_us.shape[0] + 1)
turbines_us.shape

turbines_us = turbines_us[turbines_pred['prediction'] > 0.99]
turbines_us = turbines_us[(~turbines_us['t_hh'].isna())]
turbines_us = turbines_us[~turbines_us['t_cap'].isna()]
turbines_us = turbines_us[~turbines_us['t_rd'].isna()]


/home/jschmidt/.conda/envs/windturbine-identification/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [8]:
turbines_us.head()
turbines_us[turbines_us['filenames'] == '3598.tif'][['t_cap','t_hh','t_rd']]

,t_cap,t_hh,t_rd
3597,65.0,24.6,16.0


In [9]:
nmb_samples_all = turbines_us.shape[0]

#turbines_us = turbines_us[turbines_us['id'] <= nmb_samples_all]
#turbines_us.shape

In [10]:
share_train = nmb_samples_all * 0.8
share_train

turbines_us

,case_id,faa_ors,faa_asn,usgs_pr_id,t_state,t_county,t_fips,p_name,p_year,p_tnum,...,t_img_date,t_img_srce,xlong,ylat,KW,Long,Lat,prediction,filenames,id
5,3000010,NaN,NaN,16102.0,CA,Riverside County,6065,Alta Mesa 1 and 2,1988.0,115,...,11/3/2018,Digital Globe,-116.666191,33.944595,160.0,-116.666191,33.944595,1.000000,6.tif,6
23,3000075,NaN,NaN,2744.0,CA,Kern County,6029,Sky River,1990.0,342,...,11/28/2017,Digital Globe,-118.238991,35.240494,225.0,-118.238991,35.240494,0.999992,24.tif,24
25,3000078,NaN,NaN,3846.0,CA,Kern County,6029,Oasis Power Partners,2004.0,60,...,1/1/2012,NAIP,-118.296288,35.067696,1000.0,-118.296288,35.067696,1.000000,26.tif,26
29,3000086,NaN,NaN,15221.0,CA,Riverside County,6065,San Gorgonio Farms Wind Farm,2008.0,112,...,10/15/2017,Digital Globe,-116.630142,33.923340,500.0,-116.630142,33.923340,0.999998,30.tif,30
30,3000088,NaN,NaN,11050.0,CA,Alameda County,6001,Patterson Pass,1985.0,201,...,10/16/2018,Digital Globe,-121.611084,37.691296,65.0,-121.611084,37.691296,1.000000,31.tif,31
32,3000090,NaN,NaN,11091.0,CA,Alameda County,6001,Patterson Wind,1985.0,73,...,10/16/2018,Digital Globe,-121.614293,37.696745,65.0,-121.614293,37.696745,1.000000,33.tif,33
38,3000102,NaN,NaN,20799.0,IA,Winnebago County,19189,Forest City High School,1999.0,1,...,3/20/2015,Google Earth,-93.653793,43.266293,600.0,-93.653793,43.266293,0.999961,39.tif,39
41,3000110,NaN,NaN,14352.0,CA,Riverside County,6065,San Gorgonio Westwinds II LLC,1999.0,48,...,7/28/2017,Digital Globe,-116.573486,33.905193,700.0,-116.573486,33.905193,1.000000,42.tif,42
53,3000137,NaN,NaN,44088.0,TX,Crockett County,48105,Southwest Mesa,1999.0,106,...,1/9/2018,Digital Globe,-102.119492,31.078196,700.0,-102.119492,31.078196,1.000000,54.tif,54
54,3000138,NaN,NaN,16090.0,CA,Riverside County,6065,Alta Mesa 3,1995.0,42,...,11/3/2018,Digital Globe,-116.666227,33.946156,225.0,-116.666227,33.946156,1.000000,55.tif,55


In [11]:
cnt = 0
flag = False

for i in range(0, nmb_samples_all):
    
        
        
    #print(file)
    file_s = str(turbines_us.iloc[[i]]['id'].values[0]) + '.tif'
    file_d = str(turbines_us.iloc[[i]]['id'].values[0]) + '.png'
    
    #print(file)
        
    src = os.path.join(src_dir_tb, file_s)
        
    if(cnt % 1000 == 0):
        print(cnt)
        print(src)
            
    #print(src)
        
        
        
    if(os.path.isfile(src)):
            
        dst = ""
        
        if(cnt < share_train):
           
            dst = os.path.join(train_dir, file_d)
            
        if(cnt > share_train):
                
            
            if(not flag):
                print("test files here")
                print(dst)
            
            dst = os.path.join(test_dir, file_d)
            
            if(not flag):
                print(dst)

        fpr.copyfile_to_png(src, dst)
        cnt += 1
    else:
        print("no")
        print(src)
            


0
data/aerialImages/GOOGLE/RESOLUTION19/US/raw/Turbines/6.tif
1000
data/aerialImages/GOOGLE/RESOLUTION19/US/raw/Turbines/4520.tif
2000
data/aerialImages/GOOGLE/RESOLUTION19/US/raw/Turbines/5563.tif
3000
data/aerialImages/GOOGLE/RESOLUTION19/US/raw/Turbines/6573.tif
4000
data/aerialImages/GOOGLE/RESOLUTION19/US/raw/Turbines/7589.tif
5000
data/aerialImages/GOOGLE/RESOLUTION19/US/raw/Turbines/8597.tif
6000
data/aerialImages/GOOGLE/RESOLUTION19/US/raw/Turbines/9597.tif
7000
data/aerialImages/GOOGLE/RESOLUTION19/US/raw/Turbines/10605.tif
8000
data/aerialImages/GOOGLE/RESOLUTION19/US/raw/Turbines/11610.tif
9000
data/aerialImages/GOOGLE/RESOLUTION19/US/raw/Turbines/12614.tif
10000
data/aerialImages/GOOGLE/RESOLUTION19/US/raw/Turbines/13618.tif
11000
data/aerialImages/GOOGLE/RESOLUTION19/US/raw/Turbines/14623.tif
12000
data/aerialImages/GOOGLE/RESOLUTION19/US/raw/Turbines/15630.tif
13000
data/aerialImages/GOOGLE/RESOLUTION19/US/raw/Turbines/16634.tif
14000
data/aerialImages/GOOGLE/RESOLUTION19

test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41606.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41607.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41608.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41609.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41610.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41611.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41612.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41613.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41614.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41615.png
test files here

data/aerialImages/GOOGLE/RESOLUTI

test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41774.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41775.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41776.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41777.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41778.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41779.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41780.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41781.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41782.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41783.png
test files here

data/aerialImages/GOOGLE/RESOLUTI

test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41869.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41870.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41871.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41872.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41873.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41874.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41875.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41876.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41879.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41884.png
test files here

data/aerialImages/GOOGLE/RESOLUTI

test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41970.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41971.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41972.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41973.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41974.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41975.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41976.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41977.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41978.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/41979.png
test files here

data/aerialImages/GOOGLE/RESOLUTI

...

test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/58909.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/58910.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/58911.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/58912.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/58913.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/58914.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/58915.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/58916.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/58917.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/58918.png
test files here

data/aerialImages/GOOGLE/RESOLUTI

test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/58997.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/58998.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/58999.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59000.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59001.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59002.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59003.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59004.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59005.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59006.png
test files here

data/aerialImages/GOOGLE/RESOLUTI

test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59086.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59087.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59088.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59089.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59090.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59091.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59092.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59093.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59094.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59095.png
test files here

data/aerialImages/GOOGLE/RESOLUTI

test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59176.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59177.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59178.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59179.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59180.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59181.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59182.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59183.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59184.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59185.png
test files here

data/aerialImages/GOOGLE/RESOLUTI

test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59583.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59588.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59592.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59598.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59605.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59613.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59614.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59615.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59631.png
test files here

data/aerialImages/GOOGLE/RESOLUTION18/US/type_keras_regression/test/59634.png
test files here

data/aerialImages/GOOGLE/RESOLUTI

In [12]:
print('total training turbine images ', len(os.listdir(train_dir)))
print('total testing turbine images ', len(os.listdir(test_dir)))

pd_out = turbines_us[['id','t_hh']]

pd_out['id_s'] = turbines_us['id'].map(str) + '.png'
pd_out['t_hh'] = turbines_us['t_hh']
pd_out['t_rd'] = turbines_us['t_rd']
pd_out['t_cap'] = turbines_us['t_cap']

share_train_i = len(os.listdir(train_dir))

pd_out_train = pd_out.iloc[0: share_train_i]
pd_out_test = pd_out.iloc[((share_train_i) + 1) : pd_out.shape[0]]
                                      
pd_out_train.to_csv(train_dir + "list.csv")
pd_out_test.to_csv(test_dir + "list.csv")
pd_out

total training turbine images  36518
total testing turbine images  9129


/home/jschmidt/.conda/envs/windturbine-identification/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/jschmidt/.conda/envs/windturbine-identification/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/jschmidt/.conda/envs/windturbine-identification/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

,id,t_hh,id_s,t_rd,t_cap
5,6,23.0,6.png,23.0,160.0
23,24,30.0,24.png,27.0,225.0
25,26,60.0,26.png,61.4,1000.0
29,30,30.0,30.png,39.0,500.0
30,31,24.6,31.png,16.0,65.0
32,33,24.0,33.png,16.0,65.0
38,39,50.0,39.png,43.0,600.0
41,42,50.0,42.png,48.0,700.0
53,54,50.0,54.png,48.0,700.0
54,55,40.0,55.png,27.0,225.0


In [13]:
print(len(os.listdir(train_dir)))
print(pd_out_train.shape)
print(share_train)
print(share_train_i)



36519
(36518, 5)
36517.6
36518
